In [77]:
!pip install numpy pandas
import numpy as np
import pandas as pd

def relu(x):
    return np.maximum(0,x)

def drelu(x):
    a = np.zeros(x.shape)
    for i in range(x.shape[0]):
        if x[i]>0: a[i]=1
        else: a[i] = 0
    return a

def oneHotEncode(y, nc):
    y_encoded = np.zeros((1,nc))
    y_encoded[0,y] = 1
    y_encoded = y_encoded.reshape((y_encoded.shape[1],))
    return y_encoded

def normalize(x, min, max):
    x /= (max-min)
    return x.reshape((x.shape[0],))

def softmax(x):
    a = np.exp(x)/sum(np.exp(x))
    return a


In [92]:
df = pd.read_csv("MNIST_ORG/mnist_train.csv", dtype = float)
nf = np.array(df)

np.random.seed(43)
np.random.shuffle(nf)

fMM = nf[:,1:]
dVV = nf[:,0]

testset = pd.read_csv("MNIST_ORG/mnist_test.csv", dtype = float)
fMM_test = testset.iloc[:,1:].values
dVV_test = testset.iloc[:,0].values

w1 = np.random.randn(16,784)
b1 = np.random.randn(16,)

w2 = np.random.randn(16,16)
b2 = np.random.randn(16,)

w3 = np.random.rand(10,16)
b3 = np.random.randn(10,)


In [ ]:
def cost(z3, y):
    y_en = oneHotEncode(y, 10)
    return -np.sum(y_en * np.log(z3 + 1e-9))

def forward_propagate(x, w1, b1, w2, b2, w3, b3):
    x = normalize(x, 0, 255)
    a1 = np.dot(w1, x) + b1
    z1 = relu(a1)
    a2 = np.dot(w2, z1) + b2
    z2 = relu(a2)
    a3 = np.dot(w3, z2) + b3
    z3 = softmax(a3)
    return a1, a2, a3, z1, z2, z3

def back_propagate(x, y, w1, b1, w2, b2, w3, b3, a1, a2, a3, z1, z2, z3, lrate):
    y_en = oneHotEncode(y, 10)

    da3 = z3 - y_en
    da2 = np.dot(w3.T, da3) * drelu(a2)
    da1 = np.dot(w2.T, da2) * drelu(a1)

    dw3 = np.dot(da3.reshape((da3.shape[0], 1)), z2.reshape((1, z2.shape[0])))
    dw2 = np.dot(da2.reshape((da2.shape[0], 1)), z1.reshape((1, z1.shape[0])))
    dw1 = np.dot(da1.reshape((da1.shape[0], 1)), x.reshape((1, x.shape[0])))

    w3 -= lrate * dw3
    b3 -= lrate * da3
    w2 -= lrate * dw2
    b2 -= lrate * da2
    w1 -= lrate * dw1
    b1 -= lrate * da1

    return w1, b1, w2, b2, w3, b3

def train_singlet(x, y, w1, b1, w2, b2, w3, b3, lrate):
    a1, a2, a3, z1, z2, z3 = forward_propagate(x, w1, b1, w2, b2, w3, b3)
    print("Cost:", cost(z3, y))
    return back_propagate(x, y, w1, b1, w2, b2, w3, b3, a1, a2, a3, z1, z2, z3, lrate)

def train_model(fMM, dVV, w1, b1, w2, b2, w3, b3, lrate):
    for i in range(fMM.shape[0]):
        x = fMM[i, :]
        y = int(dVV[i])
        w1, b1, w2, b2, w3, b3 = train_singlet(x, y, w1, b1, w2, b2, w3, b3, lrate)
    return w1, b1, w2, b2, w3, b3

w1, b1, w2, b2, w3, b3 = train_model(fMM, dVV, w1, b1, w2, b2, w3, b3, 0.01)
print("Training complete.")

def calc_acc(fMM_test, dVV_test, w1, b1, w2, b2, w3, b3):
    corr =0
    for i in range(fMM_test.shape[0]):
        x = fMM_test[i,:]
        y = int(dVV_test[i])
        a1, a2, a3, z1, z2, z3 = forward_propagate(x, w1, b1, w2, b2, w3, b3)
        pred = np.argmax(z3)
        if pred == y:
            corr += 1
    return corr / fMM_test.shape[0]

print("Accuracy:", calc_acc(fMM_test, dVV_test, w1, b1, w2, b2, w3, b3))

Cost: 20.723265836583817
Cost: 0.0005719886143180869
Cost: 20.72326583694641
Cost: 20.72326583694503
Cost: 20.72326583694641
Cost: 20.72326583694641
Cost: 15.825589102884981
Cost: 20.72326583666248
Cost: 20.72326247329012
Cost: 20.72326583694641
Cost: 0.30440084369072
Cost: 20.702959489043284
Cost: 14.798498531647866
Cost: 17.735445377472644
Cost: 13.885496063414283
Cost: 20.722850918377993
Cost: 2.7425994749006703e-06
Cost: 20.72325639108265
Cost: 20.723195235126063
Cost: 8.187327410250184
Cost: 0.8246422476770469
Cost: 12.211216297017243
Cost: 20.723265836945853
Cost: 13.283964008023025
Cost: 16.485035913679674
Cost: 7.8503823222759825
Cost: 2.7945634348672934
Cost: 1.9434613137942063
Cost: 11.178385620402025
Cost: 0.001409391491804549
Cost: 14.119452375415392
Cost: 13.743458192484821
Cost: 4.724876918879749
Cost: 0.0026414012084442925
Cost: 20.723072520896427
Cost: 11.266997724909642
Cost: 2.3828359649735504
Cost: 5.633951138591316
Cost: 10.20457904471276
Cost: 0.03631134555079754
C